# Comprensión y Análisis Exploratorio de Datos (EDA)

**Proyecto:** Pipeline MLOps - Predicción de Pago a Tiempo de Créditos

**Autor:** Alexis Jacquet

**Fecha:** 5 de febrero de 2026

---

## Objetivo

Realizar un análisis exploratorio exhaustivo de los datos para:
1. Comprender la estructura y características de los datos
2. Identificar patrones, tendencias y anomalías
3. Detectar problemas de calidad de datos
4. Establecer reglas de validación
5. Identificar transformaciones necesarias para el modelado

## Contenido

1. Carga de Datos y Configuración
2. Exploración Inicial de Datos
3. Análisis Univariable
4. Análisis Bivariable
5. Análisis Multivariable
6. Reglas de Validación
7. Transformaciones Identificadas
8. Conclusiones y Recomendaciones

## 1. Carga de Datos y Configuración

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr, spearmanr

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

np.random.seed(42)

print("✓ Librerías cargadas y configuradas")

In [ ]:
# Cargar datos
RUTA_RAIZ = Path.cwd().parent.parent
RUTA_DATOS = RUTA_RAIZ / 'Base_de_datos.csv'

df = pd.read_csv(RUTA_DATOS)
print(f"✓ Datos cargados: {df.shape[0]:,} filas x {df.shape[1]} columnas")

# Crear una copia para trabajar sin modificar el original
df_original = df.copy()
print("✓ Copia de seguridad creada")

## 2. Exploración Inicial de Datos

### 2.1 Descripción General

In [ ]:
print("="*80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("="*80)
print(f"\nDimensiones: {df.shape[0]:,} registros x {df.shape[1]} variables")
print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n" + "="*80)

In [ ]:
# Vista previa de los datos
print("Primeras 10 filas del dataset:")
df.head(10)

In [ ]:
# Información detallada de columnas
df.info()

### 2.2 Caracterización de Variables

Clasificaremos las variables según su tipo y naturaleza para un análisis más estructurado.

In [ ]:
# Identificar tipos de variables
columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
columnas_categoricas = df.select_dtypes(include=['object']).columns.tolist()
columnas_fecha = [col for col in df.columns if 'fecha' in col.lower()]

# Variable objetivo
variable_objetivo = 'Pago_atiempo'

# Eliminar variable objetivo y fechas de las listas
if variable_objetivo in columnas_numericas:
    columnas_numericas.remove(variable_objetivo)
for fecha in columnas_fecha:
    if fecha in columnas_numericas:
        columnas_numericas.remove(fecha)
    if fecha in columnas_categoricas:
        columnas_categoricas.remove(fecha)

print("="*80)
print("CARACTERIZACIÓN DE VARIABLES")
print("="*80)
print(f"\n📊 VARIABLE OBJETIVO:")
print(f"   {variable_objetivo}")
print(f"\n🔢 VARIABLES NUMÉRICAS ({len(columnas_numericas)}):")
for i, col in enumerate(columnas_numericas, 1):
    print(f"   {i:2d}. {col}")
print(f"\n📝 VARIABLES CATEGÓRICAS ({len(columnas_categoricas)}):")
for i, col in enumerate(columnas_categoricas, 1):
    print(f"   {i:2d}. {col}")
print(f"\n📅 VARIABLES DE FECHA ({len(columnas_fecha)}):")
for i, col in enumerate(columnas_fecha, 1):
    print(f"   {i:2d}. {col}")
print("\n" + "="*80)

### 2.3 Análisis de Valores Nulos

Identificaremos y cuantificaremos los valores nulos en cada columna.

In [ ]:
# Análisis detallado de valores nulos
def analizar_nulos(dataframe):
    """
    Analiza valores nulos en el dataframe
    """
    nulos_count = dataframe.isnull().sum()
    nulos_pct = (nulos_count / len(dataframe)) * 100
    
    resumen = pd.DataFrame({
        'Columna': dataframe.columns,
        'Tipo_Dato': dataframe.dtypes,
        'Valores_Nulos': nulos_count.values,
        'Porcentaje_Nulos': nulos_pct.values,
        'Valores_Unicos': [dataframe[col].nunique() for col in dataframe.columns]
    })
    
    resumen = resumen.sort_values('Valores_Nulos', ascending=False)
    return resumen

resumen_nulos = analizar_nulos(df)

print("="*80)
print("ANÁLISIS DE VALORES NULOS")
print("="*80)
print(f"\nTotal de valores nulos en el dataset: {df.isnull().sum().sum():,}")
print(f"Porcentaje total de nulos: {(df.isnull().sum().sum() / df.size * 100):.2f}%\n")

# Mostrar solo columnas con nulos
resumen_con_nulos = resumen_nulos[resumen_nulos['Valores_Nulos'] > 0]

if len(resumen_con_nulos) > 0:
    print(f"\nColumnas con valores nulos: {len(resumen_con_nulos)}\n")
    print(resumen_con_nulos.to_string(index=False))
    
    # Visualización
    if len(resumen_con_nulos) > 0:
        plt.figure(figsize=(12, 6))
        plt.barh(resumen_con_nulos['Columna'], resumen_con_nulos['Porcentaje_Nulos'], 
                color='coral', edgecolor='black')
        plt.xlabel('Porcentaje de Valores Nulos (%)', fontsize=11)
        plt.title('Distribución de Valores Nulos por Columna', fontsize=13, fontweight='bold')
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print("\n✓ No se encontraron valores nulos en ninguna columna")

### 2.4 Unificar Representación de Valores Nulos

Convertiremos diferentes representaciones de nulos a un formato estándar.

In [ ]:
# Valores que deben considerarse como nulos
valores_nulos = ['', ' ', 'NA', 'N/A', 'na', 'n/a', 'NULL', 'null', 'None', 'none', '-', '--', '?']

print("Unificando representaciones de valores nulos...")
print(f"\nValores tratados como nulos: {valores_nulos}")

# Reemplazar valores nulos en columnas categóricas
for col in columnas_categoricas:
    df[col] = df[col].replace(valores_nulos, np.nan)

# Verificar cambios
nulos_despues = df.isnull().sum().sum()
print(f"\nTotal de nulos después de unificar: {nulos_despues:,}")
print("✓ Valores nulos unificados")

### 2.5 Conversión de Tipos de Datos

Aseguraremos que cada columna tenga el tipo de dato correcto.

In [ ]:
print("="*80)
print("CONVERSIÓN DE TIPOS DE DATOS")
print("="*80)

# Convertir columnas de fecha
for col in columnas_fecha:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
            print(f"✓ {col} convertida a datetime")
        except Exception as e:
            print(f"❌ Error al convertir {col}: {str(e)}")

# Verificar y convertir variable objetivo a entero
if variable_objetivo in df.columns:
    df[variable_objetivo] = df[variable_objetivo].astype(int)
    print(f"✓ {variable_objetivo} confirmada como int")

# Variables categóricas que podrían ser booleanas o nominales
# tipo_credito: parece ser categórica ordinal o nominal
# tipo_laboral: categórica nominal
# tendencia_ingresos: categórica ordinal

print("\nTipos de datos actualizados:")
print(df.dtypes)

### 2.6 Identificación de Variables Irrelevantes

Analizaremos si existen variables que no aportan información al modelo.

In [ ]:
print("="*80)
print("ANÁLISIS DE VARIABLES IRRELEVANTES")
print("="*80)

variables_baja_varianza = []
variables_muchos_nulos = []
variables_constantes = []

for col in df.columns:
    # Variables con un solo valor único (constantes)
    if df[col].nunique() == 1:
        variables_constantes.append(col)
    
    # Variables con más del 90% de nulos
    pct_nulos = (df[col].isnull().sum() / len(df)) * 100
    if pct_nulos > 90:
        variables_muchos_nulos.append(col)
    
    # Variables con muy baja varianza (>95% mismo valor para categóricas)
    if col in columnas_categoricas:
        if df[col].value_counts(normalize=True).iloc[0] > 0.95:
            variables_baja_varianza.append(col)

print(f"\nVariables constantes (1 valor único): {len(variables_constantes)}")
if variables_constantes:
    print(f"   {variables_constantes}")

print(f"\nVariables con >90% nulos: {len(variables_muchos_nulos)}")
if variables_muchos_nulos:
    print(f"   {variables_muchos_nulos}")

print(f"\nVariables con baja varianza (>95% mismo valor): {len(variables_baja_varianza)}")
if variables_baja_varianza:
    for var in variables_baja_varianza:
        print(f"   - {var}: {df[var].value_counts(normalize=True).iloc[0]*100:.1f}% es '{df[var].value_counts().index[0]}'")

variables_a_eliminar = list(set(variables_constantes + variables_muchos_nulos))

if len(variables_a_eliminar) > 0:
    print(f"\n⚠️  Se recomienda eliminar {len(variables_a_eliminar)} variables")
    print(f"    Variables: {variables_a_eliminar}")
else:
    print("\n✓ No se identificaron variables claramente irrelevantes")

## 3. Análisis Univariable

### 3.1 Variables Numéricas

In [ ]:
# Estadísticas descriptivas completas para variables numéricas
print("="*80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("="*80)
df[columnas_numericas].describe().T

In [ ]:
# Estadísticas adicionales: skewness, kurtosis
print("="*80)
print("MEDIDAS DE FORMA DE DISTRIBUCIÓN")
print("="*80)

estadisticas_forma = pd.DataFrame({
    'Variable': columnas_numericas,
    'Skewness': [df[col].skew() for col in columnas_numericas],
    'Kurtosis': [df[col].kurtosis() for col in columnas_numericas]
})

# Interpretación de skewness
estadisticas_forma['Interpretacion_Skewness'] = estadisticas_forma['Skewness'].apply(
    lambda x: 'Simétrica' if abs(x) < 0.5 else ('Asimétrica derecha' if x > 0 else 'Asimétrica izquierda')
)

print(estadisticas_forma.to_string(index=False))

In [ ]:
# Visualización: Histogramas y boxplots para variables numéricas
def plot_distribucion_numerica(df, columnas, filas=4, columnas_grafico=3):
    """
    Crea histogramas y boxplots para variables numéricas
    """
    n_cols = len(columnas)
    n_filas = (n_cols // columnas_grafico) + (1 if n_cols % columnas_grafico > 0 else 0)
    
    fig, axes = plt.subplots(n_filas, columnas_grafico, figsize=(18, n_filas * 4))
    axes = axes.flatten() if n_cols > 1 else [axes]
    
    for idx, col in enumerate(columnas):
        # Histograma con KDE
        axes[idx].hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7, density=True)
        df[col].dropna().plot(kind='kde', ax=axes[idx], color='red', linewidth=2)
        axes[idx].set_title(f'Distribución: {col}', fontweight='bold')
        axes[idx].set_xlabel('')
        axes[idx].grid(alpha=0.3)
    
    # Ocultar ejes vacíos
    for idx in range(n_cols, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("Distribuciones de Variables Numéricas:")
plot_distribucion_numerica(df, columnas_numericas[:9])  # Primeras 9 variables

In [ ]:
# Continuar con más variables si hay más de 9
if len(columnas_numericas) > 9:
    plot_distribucion_numerica(df, columnas_numericas[9:])

In [ ]:
# Boxplots para detectar outliers
def plot_boxplots(df, columnas, filas=4, columnas_grafico=3):
    """
    Crea boxplots para variables numéricas
    """
    n_cols = len(columnas)
    n_filas = (n_cols // columnas_grafico) + (1 if n_cols % columnas_grafico > 0 else 0)
    
    fig, axes = plt.subplots(n_filas, columnas_grafico, figsize=(18, n_filas * 3))
    axes = axes.flatten() if n_cols > 1 else [axes]
    
    for idx, col in enumerate(columnas):
        axes[idx].boxplot(df[col].dropna(), vert=True)
        axes[idx].set_title(f'Boxplot: {col}', fontweight='bold')
        axes[idx].set_ylabel(col)
        axes[idx].grid(alpha=0.3)
    
    # Ocultar ejes vacíos
    for idx in range(n_cols, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("\nBoxplots para Detección de Outliers:")
plot_boxplots(df, columnas_numericas[:9])

In [ ]:
if len(columnas_numericas) > 9:
    plot_boxplots(df, columnas_numericas[9:])

In [ ]:
# Análisis de outliers usando IQR
def analizar_outliers(df, columnas):
    """
    Detecta outliers usando el método IQR
    """
    resultados = []
    
    for col in columnas:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)][col]
        
        resultados.append({
            'Variable': col,
            'Q1': Q1,
            'Q3': Q3,
            'IQR': IQR,
            'Limite_Inferior': limite_inferior,
            'Limite_Superior': limite_superior,
            'N_Outliers': len(outliers),
            'Porcentaje_Outliers': (len(outliers) / len(df)) * 100
        })
    
    return pd.DataFrame(resultados)

print("="*80)
print("ANÁLISIS DE OUTLIERS (MÉTODO IQR)")
print("="*80)
outliers_df = analizar_outliers(df, columnas_numericas)
print(outliers_df.to_string(index=False))

### 3.2 Variables Categóricas

In [ ]:
# Estadísticas descriptivas para variables categóricas
print("="*80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES CATEGÓRICAS")
print("="*80)

for col in columnas_categoricas:
    print(f"\n{'='*60}")
    print(f"Variable: {col}")
    print(f"{'='*60}")
    print(f"Valores únicos: {df[col].nunique()}")
    print(f"Valor más frecuente: {df[col].mode()[0] if len(df[col].mode()) > 0 else 'N/A'}")
    print(f"\nDistribución de frecuencias:")
    
    frecuencias = df[col].value_counts()
    frecuencias_pct = df[col].value_counts(normalize=True) * 100
    
    resumen = pd.DataFrame({
        'Frecuencia': frecuencias,
        'Porcentaje': frecuencias_pct
    })
    
    print(resumen.head(10))  # Mostrar top 10 categorías

In [ ]:
# Visualización de variables categóricas
def plot_categoricas(df, columnas):
    """
    Crea gráficos de barras para variables categóricas
    """
    for col in columnas:
        plt.figure(figsize=(12, 5))
        
        # Countplot
        value_counts = df[col].value_counts()
        plt.subplot(1, 2, 1)
        value_counts.plot(kind='bar', edgecolor='black', alpha=0.7)
        plt.title(f'Distribución: {col}', fontweight='bold', fontsize=12)
        plt.xlabel(col)
        plt.ylabel('Frecuencia')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        
        # Pie chart
        plt.subplot(1, 2, 2)
        if len(value_counts) <= 10:  # Solo si hay pocas categorías
            plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90)
            plt.title(f'Proporción: {col}', fontweight='bold', fontsize=12)
        else:
            top10 = value_counts.head(10)
            plt.pie(top10, labels=top10.index, autopct='%1.1f%%', startangle=90)
            plt.title(f'Proporción (Top 10): {col}', fontweight='bold', fontsize=12)
        
        plt.tight_layout()
        plt.show()

print("Distribuciones de Variables Categóricas:")
plot_categoricas(df, columnas_categoricas)

## 4. Análisis Bivariable

### 4.1 Relación con la Variable Objetivo

In [ ]:
# Distribución de la variable objetivo
print("="*80)
print("ANÁLISIS DE LA VARIABLE OBJETIVO: Pago_atiempo")
print("="*80)

distribucion_objetivo = df[variable_objetivo].value_counts().sort_index()
distribucion_pct = df[variable_objetivo].value_counts(normalize=True).sort_index() * 100

print(f"\nDistribución:")
print(f"  Clase 0 (No pagó a tiempo): {distribucion_objetivo[0]:,} ({distribucion_pct[0]:.2f}%)")
print(f"  Clase 1 (Pagó a tiempo): {distribucion_objetivo[1]:,} ({distribucion_pct[1]:.2f}%)")

ratio = min(distribucion_objetivo) / max(distribucion_objetivo)
print(f"\nRatio de balance: {ratio:.3f}")

if ratio < 0.5:
    print("⚠️  Dataset desbalanceado - Considerar técnicas de balanceo")
else:
    print("✓ Dataset razonablemente balanceado")

In [ ]:
# Variables numéricas vs Variable objetivo
def analizar_numerica_vs_objetivo(df, columnas_num, var_objetivo):
    """
    Analiza la relación entre variables numéricas y la variable objetivo
    """
    resultados = []
    
    for col in columnas_num:
        # Estadísticas por clase
        clase_0 = df[df[var_objetivo] == 0][col]
        clase_1 = df[df[var_objetivo] == 1][col]
        
        # Test estadístico (t-test)
        try:
            t_stat, p_value = stats.ttest_ind(clase_0.dropna(), clase_1.dropna())
        except:
            t_stat, p_value = np.nan, np.nan
        
        resultados.append({
            'Variable': col,
            'Media_Clase_0': clase_0.mean(),
            'Media_Clase_1': clase_1.mean(),
            'Mediana_Clase_0': clase_0.median(),
            'Mediana_Clase_1': clase_1.median(),
            'Diferencia_Medias': abs(clase_0.mean() - clase_1.mean()),
            'p_value': p_value,
            'Significativa': 'Sí' if p_value < 0.05 else 'No'
        })
    
    return pd.DataFrame(resultados).sort_values('p_value')

print("="*80)
print("ANÁLISIS BIVARIABLE: VARIABLES NUMÉRICAS VS OBJETIVO")
print("="*80)
analisis_num_objetivo = analizar_numerica_vs_objetivo(df, columnas_numericas, variable_objetivo)
print(analisis_num_objetivo.to_string(index=False))

In [ ]:
# Visualización: Boxplots por clase objetivo
def plot_boxplots_por_clase(df, columnas_num, var_objetivo, n_vars=6):
    """
    Crea boxplots comparando distribuciones por clase objetivo
    """
    # Seleccionar variables más significativas
    vars_significativas = analisis_num_objetivo.head(n_vars)['Variable'].tolist()
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for idx, col in enumerate(vars_significativas):
        df.boxplot(column=col, by=var_objetivo, ax=axes[idx])
        axes[idx].set_title(f'{col} por Pago_atiempo', fontweight='bold')
        axes[idx].set_xlabel('Pago a tiempo')
        axes[idx].set_ylabel(col)
        plt.sca(axes[idx])
        plt.xticks([1, 2], ['No (0)', 'Sí (1)'])
    
    plt.suptitle('')  # Eliminar título automático de pandas
    plt.tight_layout()
    plt.show()

print("\nBoxplots de Variables Más Significativas por Clase:")
plot_boxplots_por_clase(df, columnas_numericas, variable_objetivo)

In [ ]:
# Variables categóricas vs Variable objetivo
def analizar_categorica_vs_objetivo(df, columnas_cat, var_objetivo):
    """
    Analiza la relación entre variables categóricas y la variable objetivo
    """
    resultados = []
    
    for col in columnas_cat:
        # Tabla de contingencia
        tabla_contingencia = pd.crosstab(df[col], df[var_objetivo])
        
        # Test Chi-cuadrado
        try:
            chi2, p_value, dof, expected = chi2_contingency(tabla_contingencia)
        except:
            chi2, p_value = np.nan, np.nan
        
        # Cramér's V (medida de asociación)
        n = tabla_contingencia.sum().sum()
        cramers_v = np.sqrt(chi2 / (n * (min(tabla_contingencia.shape) - 1))) if not np.isnan(chi2) else np.nan
        
        resultados.append({
            'Variable': col,
            'Categorias_Unicas': df[col].nunique(),
            'Chi2': chi2,
            'p_value': p_value,
            'Cramers_V': cramers_v,
            'Significativa': 'Sí' if p_value < 0.05 else 'No'
        })
    
    return pd.DataFrame(resultados).sort_values('p_value')

if len(columnas_categoricas) > 0:
    print("\n" + "="*80)
    print("ANÁLISIS BIVARIABLE: VARIABLES CATEGÓRICAS VS OBJETIVO")
    print("="*80)
    analisis_cat_objetivo = analizar_categorica_vs_objetivo(df, columnas_categoricas, variable_objetivo)
    print(analisis_cat_objetivo.to_string(index=False))

In [ ]:
# Visualización: Gráficos de barras agrupadas
if len(columnas_categoricas) > 0:
    for col in columnas_categoricas:
        plt.figure(figsize=(12, 5))
        
        # Tabla de contingencia normalizada
        tabla = pd.crosstab(df[col], df[variable_objetivo], normalize='index') * 100
        
        tabla.plot(kind='bar', stacked=False, edgecolor='black', alpha=0.7)
        plt.title(f'{col} vs Pago_atiempo', fontweight='bold', fontsize=13)
        plt.xlabel(col)
        plt.ylabel('Porcentaje (%)')
        plt.legend(['No pagó (0)', 'Pagó (1)'], title='Pago a tiempo')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()

## 5. Análisis Multivariable

### 5.1 Matriz de Correlación

In [ ]:
# Calcular matriz de correlación
print("="*80)
print("MATRIZ DE CORRELACIÓN - VARIABLES NUMÉRICAS")
print("="*80)

# Incluir variable objetivo en la correlación
cols_para_corr = columnas_numericas + [variable_objetivo]
correlacion = df[cols_para_corr].corr()

# Visualización de la matriz de correlación
plt.figure(figsize=(16, 14))
mask = np.triu(np.ones_like(correlacion, dtype=bool))  # Máscara para triángulo superior
sns.heatmap(correlacion, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación de Variables Numéricas', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Identificar correlaciones fuertes
print("\n" + "="*80)
print("CORRELACIONES FUERTES (|r| > 0.7)")
print("="*80)

# Obtener correlaciones significativas (excluyendo diagonal)
correlaciones_fuertes = []
for i in range(len(correlacion.columns)):
    for j in range(i+1, len(correlacion.columns)):
        if abs(correlacion.iloc[i, j]) > 0.7:
            correlaciones_fuertes.append({
                'Variable_1': correlacion.columns[i],
                'Variable_2': correlacion.columns[j],
                'Correlacion': correlacion.iloc[i, j]
            })

if correlaciones_fuertes:
    df_corr_fuertes = pd.DataFrame(correlaciones_fuertes).sort_values('Correlacion', 
                                                                        key=abs, ascending=False)
    print(df_corr_fuertes.to_string(index=False))
    print("\n⚠️  Advertencia: Variables con correlación muy alta pueden causar multicolinealidad")
else:
    print("No se encontraron correlaciones fuertes (|r| > 0.7)")

In [ ]:
# Correlación con la variable objetivo
print("\n" + "="*80)
print("CORRELACIÓN CON LA VARIABLE OBJETIVO")
print("="*80)

correlacion_objetivo = correlacion[variable_objetivo].drop(variable_objetivo).sort_values(
    key=abs, ascending=False
)

print(correlacion_objetivo)

# Visualización
plt.figure(figsize=(10, 8))
correlacion_objetivo.plot(kind='barh', color=['green' if x > 0 else 'red' for x in correlacion_objetivo],
                          edgecolor='black', alpha=0.7)
plt.title('Correlación de Variables con Pago_atiempo', fontsize=13, fontweight='bold')
plt.xlabel('Coeficiente de Correlación')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.2 Pairplot de Variables Clave

In [ ]:
# Seleccionar las 5 variables más correlacionadas con el objetivo
top_vars = correlacion_objetivo.head(5).index.tolist()
cols_pairplot = top_vars + [variable_objetivo]

print(f"Creando pairplot con las variables más relevantes: {top_vars}")
print("Esto puede tomar unos momentos...")

# Crear pairplot
sns.pairplot(df[cols_pairplot], hue=variable_objetivo, diag_kind='kde', 
             plot_kws={'alpha': 0.6}, height=2.5)
plt.suptitle('Pairplot de Variables Más Correlacionadas con Pago_atiempo', 
             y=1.02, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Reglas de Validación de Datos

Basándonos en el EDA, establecemos reglas de validación para el pipeline.

In [ ]:
print("="*80)
print("REGLAS DE VALIDACIÓN DE DATOS")
print("="*80)

reglas_validacion = {
    'tipo_credito': {
        'tipo': 'int',
        'rango': (df['tipo_credito'].min(), df['tipo_credito'].max()),
        'permite_nulos': False
    },
    'fecha_prestamo': {
        'tipo': 'datetime',
        'rango': (df['fecha_prestamo'].min(), df['fecha_prestamo'].max()),
        'permite_nulos': False
    },
    'capital_prestado': {
        'tipo': 'float',
        'rango': (0, df['capital_prestado'].quantile(0.99)),
        'permite_nulos': True
    },
    'plazo_meses': {
        'tipo': 'int',
        'rango': (1, 360),
        'permite_nulos': False
    },
    'edad_cliente': {
        'tipo': 'int',
        'rango': (18, 100),
        'permite_nulos': False
    },
    'tipo_laboral': {
        'tipo': 'str',
        'valores_validos': df['tipo_laboral'].dropna().unique().tolist(),
        'permite_nulos': True
    },
    'salario_cliente': {
        'tipo': 'int',
        'rango': (0, df['salario_cliente'].quantile(0.99)),
        'permite_nulos': False
    },
    'Pago_atiempo': {
        'tipo': 'int',
        'valores_validos': [0, 1],
        'permite_nulos': False
    }
}

print("\nReglas de validación definidas:")
for variable, reglas in reglas_validacion.items():
    print(f"\n{variable}:")
    for regla, valor in reglas.items():
        print(f"  - {regla}: {valor}")

print("\n✓ Reglas de validación establecidas")

## 7. Transformaciones Identificadas

Documentamos las transformaciones necesarias para la fase de Feature Engineering.

In [ ]:
print("="*80)
print("TRANSFORMACIONES IDENTIFICADAS PARA FEATURE ENGINEERING")
print("="*80)

transformaciones = {
    '1. Tratamiento de Nulos': [
        f"- tendencia_ingresos: {df['tendencia_ingresos'].isnull().sum()} nulos -> Imputar con moda o categoría 'Desconocido'",
        f"- puntaje: {df['puntaje'].isnull().sum()} nulos -> Imputar con mediana o media",
        f"- capital_prestado: {df['capital_prestado'].isnull().sum()} nulos -> Imputar con mediana"
    ],
    '2. Encoding de Variables Categóricas': [
        "- tipo_laboral: One-Hot Encoding o Label Encoding",
        "- tendencia_ingresos: Ordinal Encoding (Decreciente < Estable < Creciente)"
    ],
    '3. Escalado de Variables Numéricas': [
        "- Aplicar StandardScaler o MinMaxScaler a todas las variables numéricas",
        "- Considerar RobustScaler para variables con outliers"
    ],
    '4. Feature Engineering': [
        "- Crear 'ratio_cuota_salario': cuota_pactada / salario_cliente",
        "- Crear 'total_deuda': saldo_total + saldo_mora",
        "- Crear 'antiguedad_credito': días desde fecha_prestamo",
        "- Crear 'ratio_otros_prestamos': total_otros_prestamos / salario_cliente",
        "- Binning de edad_cliente en grupos etarios"
    ],
    '5. Tratamiento de Outliers': [
        "- Aplicar Winsorization o Capping en variables con outliers extremos",
        "- Considerar transformaciones logarítmicas para variables asimétricas"
    ],
    '6. Balanceo de Clases': [
        f"- Ratio actual: {ratio:.3f}",
        "- Técnicas recomendadas: SMOTE, RandomUnderSampler o Class Weight"
    ] if ratio < 0.7 else ["- No es crítico balancear (ratio aceptable)"]
}

for categoria, items in transformaciones.items():
    print(f"\n{categoria}")
    for item in items:
        print(f"  {item}")

print("\n" + "="*80)

## 8. Conclusiones y Recomendaciones

### 8.1 Resumen de Hallazgos

In [ ]:
print("="*80)
print("CONCLUSIONES DEL ANÁLISIS EXPLORATORIO")
print("="*80)

conclusiones = f"""
📊 RESUMEN DEL DATASET:
- Total de registros: {df.shape[0]:,}
- Variables numéricas: {len(columnas_numericas)}
- Variables categóricas: {len(columnas_categoricas)}
- Variables de fecha: {len(columnas_fecha)}
- Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB

🎯 VARIABLE OBJETIVO:
- Distribución: Clase 0 ({distribucion_pct[0]:.1f}%), Clase 1 ({distribucion_pct[1]:.1f}%)
- Balance: {'Desbalanceado' if ratio < 0.5 else 'Aceptable'} (ratio: {ratio:.3f})

❓ CALIDAD DE DATOS:
- Total de nulos: {df.isnull().sum().sum():,} ({(df.isnull().sum().sum() / df.size * 100):.2f}%)
- Columnas con nulos: {len(resumen_con_nulos)}
- Variables a eliminar: {len(variables_a_eliminar)}

🔍 INSIGHTS PRINCIPALES:
1. Variables más correlacionadas con Pago_atiempo:
{chr(10).join([f'   - {var}: {correlacion_objetivo[var]:.3f}' for var in correlacion_objetivo.head(5).index])}

2. Outliers detectados:
{chr(10).join([f'   - {row["Variable"]}: {row["N_Outliers"]} outliers ({row["Porcentaje_Outliers"]:.1f}%)' for _, row in outliers_df.nlargest(3, 'N_Outliers').iterrows()])}

3. Variables con alta correlación entre sí:
{chr(10).join([f'   - {row["Variable_1"]} ↔ {row["Variable_2"]}: {row["Correlacion"]:.3f}' for _, row in df_corr_fuertes.head(3).iterrows()]) if correlaciones_fuertes else '   - No se detectaron correlaciones excesivamente altas'}

✅ RECOMENDACIONES:
1. Implementar pipeline robusto de limpieza de datos
2. Aplicar técnicas de imputación para valores nulos
3. Realizar feature engineering según transformaciones identificadas
4. Considerar balanceo de clases durante el entrenamiento
5. Implementar validación de datos según reglas establecidas
6. Monitorear drift de datos en producción

✓ Dataset listo para la fase de Feature Engineering
"""

print(conclusiones)
print("="*80)

### 8.2 Exportar Resumen del EDA

In [ ]:
# Guardar resumen del EDA en archivo de texto
ruta_resumen = RUTA_RAIZ / 'eda_resumen.txt'

with open(ruta_resumen, 'w', encoding='utf-8') as f:
    f.write("RESUMEN DEL ANÁLISIS EXPLORATORIO DE DATOS\n")
    f.write("="*80 + "\n\n")
    f.write(conclusiones)
    f.write("\n\nREGLAS DE VALIDACIÓN:\n")
    f.write("-"*80 + "\n")
    for variable, reglas in reglas_validacion.items():
        f.write(f"\n{variable}:\n")
        for regla, valor in reglas.items():
            f.write(f"  - {regla}: {valor}\n")

print(f"✓ Resumen del EDA guardado en: {ruta_resumen}")
print("\n✓ Análisis Exploratorio completado exitosamente")
print("\n📍 Siguiente paso: Feature Engineering (ft_engineering.py)")